In [12]:
import palom
import numpy as np
import dask.array as da
import os
from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    OmeTiffWrapper,
    MultiImageWrapper,
)
import generate_tiff_offsets

In [20]:
!pip install generate_tiff_offsets palom numpy dask vitessce starlette==0.14.0 ujson

  Using cached scikit_image-0.19.3-cp311-cp311-macosx_15_0_arm64.whl
  Obtaining dependency information for tifffile>=2023.9.26 from https://files.pythonhosted.org/packages/d2/d7/ca95f347442e82700f591f3608e336596ee607daecbcad6a7ebd16ff5de4/tifffile-2024.7.2-py3-none-any.whl.metadata
  Using cached tifffile-2024.7.2-py3-none-any.whl.metadata (30 kB)
Using cached tifffile-2024.7.2-py3-none-any.whl (225 kB)
  Attempting uninstall: tifffile
    Found existing installation: tifffile 2025.2.18
    Uninstalling tifffile-2025.2.18:
      Successfully uninstalled tifffile-2025.2.18
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.25.2
    Uninstalling scikit-image-0.25.2:
      Successfully uninstalled scikit-image-0.25.2


In [13]:
# Data Dir
data_dir = '/Users/swarchol/Research/seal/data/astro'

image_channels = np.stack([np.load(f'{data_dir}/filter-{feature}.npy') for feature in ['u', 'g', 'r', 'i', 'z']])

segmentation_masks = np.load(f'{data_dir}/seg_masks.npy')

In [14]:
# write to temp directory with tempfile
import tempfile
with tempfile.TemporaryDirectory() as temp_dir:
    temp_dir = tempfile.mkdtemp()
    image_channels_path = os.path.join(temp_dir, 'image.ome.tif')
    segmentation_masks_path = os.path.join(temp_dir, 'mask.ome.tif')
    

In [15]:
palom.pyramid.write_pyramid(
    mosaics=[
        da.from_array(image_channels)
    ],
    output_path=image_channels_path, 
    downscale_factor=2,
    channel_names=['u', 'g', 'r', 'i', 'z']
)

2025-03-13 22:12:23.221 | INFO     | palom.pyramid:write_pyramid:166 - Writing to /var/folders/ps/x7v31pds3gn4t11djwg0t03h0000gp/T/tmp3h4q1i53/image.ome.tif


Assembling mosaic  1/ 1 (channel  1/ 5):   0%|          | 0/2 [00:00<?, ?it/s]

Assembling mosaic  1/ 1 (channel  2/ 5):   0%|          | 0/2 [00:00<?, ?it/s]

Assembling mosaic  1/ 1 (channel  3/ 5):   0%|          | 0/2 [00:00<?, ?it/s]

Assembling mosaic  1/ 1 (channel  4/ 5):   0%|          | 0/2 [00:00<?, ?it/s]

Assembling mosaic  1/ 1 (channel  5/ 5):   0%|          | 0/2 [00:00<?, ?it/s]

2025-03-13 22:12:23.368 | INFO     | palom.pyramid:write_pyramid:182 - Generating pyramid
2025-03-13 22:12:23.369 | INFO     | palom.pyramid:write_pyramid:185 -     Level 1 (1023 x 1203)
Processing channel: 100%|##########| 5/5 [00:00<00:00,  7.35it/s]
2025-03-13 22:12:24.058 | INFO     | palom.pyramid:write_pyramid:185 -     Level 2 (512 x 602)
Processing channel: 100%|##########| 5/5 [00:00<00:00, 12.51it/s]


In [16]:
# write this to a tiff file with palom
palom.pyramid.write_pyramid(
    mosaics=[
        da.from_array(segmentation_masks)
    ],
    is_mask=True,
    output_path=segmentation_masks_path, 
    downscale_factor=2,

)

2025-03-13 22:12:24.489 | INFO     | palom.pyramid:write_pyramid:166 - Writing to /var/folders/ps/x7v31pds3gn4t11djwg0t03h0000gp/T/tmp3h4q1i53/mask.ome.tif


Assembling mosaic  1/ 1 (channel  1/ 1):   0%|          | 0/2 [00:00<?, ?it/s]

2025-03-13 22:12:24.519 | INFO     | palom.pyramid:write_pyramid:182 - Generating pyramid
2025-03-13 22:12:24.519 | INFO     | palom.pyramid:write_pyramid:185 -     Level 1 (1023 x 1203)
Processing channel: 100%|##########| 1/1 [00:00<00:00, 12.49it/s]
2025-03-13 22:12:24.606 | INFO     | palom.pyramid:write_pyramid:185 -     Level 2 (512 x 602)
Processing channel: 100%|##########| 1/1 [00:00<00:00, 13.98it/s]


In [18]:
vc = VitessceConfig(
    schema_version="1.0.15",
    name="Spraggins Multi-Modal",
    description="PAS + IMS + AF From https://portal.hubmapconsortium.org/browse/collection/6a6efd0c1a2681dc7d2faab8e4ab0bca",
)
dataset = vc.add_dataset(name="Spraggins").add_object(
    MultiImageWrapper(
        image_wrappers=[
            OmeTiffWrapper(img_path=segmentation_masks_path, name="Mask"),
            OmeTiffWrapper(img_path=image_channels_path, name="Image"),
        ],
        use_physical_size_scaling=True,
    )
)
spatial = vc.add_view(cm.SPATIAL, dataset=dataset)
lc = vc.add_view(cm.LAYER_CONTROLLER, dataset=dataset).set_props(
    disableChannelsIfRgbDetected=True
)
vc.layout(spatial | (lc))

In [19]:
vw = vc.widget()
vw


VitessceWidget(config={'version': '1.0.15', 'name': 'Spraggins Multi-Modal', 'description': 'PAS + IMS + AF Fr…